# Fiction Bot IV
### A notebook based automated fiction scraper + EPub generator
This notebook is able to scrape and download all chapters from a provided internet novel url (biquge.com.cn), then auto generate a well-formatted ePub ebook, with **Table Of Contents** of course!

In [1]:
from bs4 import BeautifulSoup
import requests
import os
import shutil

In [73]:
base_url = "xbiquge.so"
book_url = "https://www.xbiquge.so/book/53005/"

In [4]:
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")
book_title = soup.h1.text
author = soup.find("meta", attrs={"property":"og:novel:author"})['content']

### Create Folder Structure for EPub
These two folders are necessary under the root directory
- META-INF
- OPS

Plus a file:
- mimetype

In [7]:
try:
    os.mkdir(f"./{book_title}")
except:
    print(f"Folder exists: ./{book_title}")
    pass

try:
    os.mkdir(f"./{book_title}/META-INF")
    os.mkdir(f"./{book_title}/OPS")
except:
    pass

Write `application/epub+zip` to the mimetype file

In [8]:
with open(f"./{book_title}/mimetype", "w") as tmp:
    tmp.write("application/epub+zip")

Create `container.xml` file

In [9]:
with open(f"./{book_title}/META-INF/container.xml", "w", encoding="utf-8") as tmp:
    tmp.write('''<?xml version="1.0" encoding="UTF-8" ?>
<container version="1.0" xmlns="urn:oasis:names:tc:opendocument:xmlns:container">
  <rootfiles>\n    <rootfile full-path="OPS/content.opf" media-type="application/oebps-package+xml"/>\n  </rootfiles>
</container>
''')

In [10]:
opfcontent = '''<?xml version="1.0" encoding="UTF-8" ?>
<package version="2.0" unique-identifier="PrimaryID" xmlns="http://www.idpf.org/2007/opf">
<metadata xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:opf="http://www.idpf.org/2007/opf">
%(metadata)s
<meta name="cover" content="cover"/>
</metadata>
<manifest>
%(manifest)s
<item id="ncx" href="content.ncx" media-type="application/x-dtbncx+xml"/>
<item id="cover" href="cover.jpg" media-type="image/jpeg"/>
</manifest>
<spine toc="ncx">
%(ncx)s
</spine>
</package>
'''
dc = '<dc:%(tag)s>%(value)s</dc:%(tag)s>'
item = "<item id='%(id)s' href='%(url)s' media-type='application/xhtml+xml'/>"
itemref = "<itemref idref='%(id)s'/>"
metadata = '\n'.join([
    dc % {'tag': 'title', 'value': book_title},
    dc % {'tag': 'creator', 'value': author},
    dc % {'tag': 'decription', 'value': "本文档由Fiction Bot IV自动生成"},
])

In [11]:
ncxcontent = '''<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE ncx PUBLIC "-//NISO//DTD ncx 2005-1//EN" "http://www.daisy.org/z3986/2005/ncx-2005-1.dtd">
<ncx version="2005-1" xmlns="http://www.daisy.org/z3986/2005/ncx/">
<head>
  <meta name="dtb:uid" content=" "/>
  <meta name="dtb:depth" content="-1"/>
  <meta name="dtb:totalPageCount" content="0"/>
  <meta name="dtb:maxPageNumber" content="0"/>
</head>
 <docTitle><text>%(title)s</text></docTitle>
 <docAuthor><text>%(creator)s</text></docAuthor>
<navMap>
%(navpoints)s
</navMap>
</ncx>
'''
navpoint = '''<navPoint id='%s' class='level1' playOrder='%d'>
<navLabel> <text>%s</text> </navLabel>
<content src='%s'/></navPoint>'''

Fetch all HTML tags of the menu entries, store in `menu_raw`

In [39]:
menu_raw = soup.find_all('dd')
menu_raw = menu_raw[12:]

Then parse href and chapter titiles from each HTML tag, store the information in `menu_info`.

In [47]:
class MenuInfo:
    
    
    def __init__(self, url, chapter_title):
        self.url = url
        self.chapter_title = chapter_title
        self.id = None
        self.epub_link = None
    
    def get_title(self):
        return self.chapter_title
    
    def get_url(self):
        return self.get_url
    
    def __str__(self):
        return f"{self.id}: {self.chapter_title} - {self.url} - {self.epub_link}"

In [54]:
menu_info = []
for index, data in enumerate(menu_raw, 1):
    try:
        m = MenuInfo(url = data.a['href'], chapter_title = data.text)
        menu_info.append(m)
    except:
        pass

# menu_preprocessing.sort(key=lambda x: x.url)
for i in range(0, 10):
    print(str(menu_info[-1-i]))


None: 新书《终末的绅士》已发布 - 39728608.html - None
None: 新书《终末的绅士》及简介 - 39714241.html - None
None: 新书预告Part.2 - 39503263.html - None
None: 新书预告Part.1 - 39396290.html - None
None: 完本感言 - 39394738.html - None
None: 第二千一百六十七章 我的细胞监狱（大结局） - 39391185.html - None
None: 第二千一百六十六章 命运合同与混沌之道 - 39375344.html - None
None: 第二千一百六十五章 线与道路 - 39375089.html - None
None: 第二千一百六十四章 工作交接 - 39374077.html - None
None: 第二千一百六十三章 本质 - 39373711.html - None


In [55]:
for c, d in enumerate(menu_info, 1):
    try:
        d.id = c
        d.epub_link = f'chapter_{c}.html'
    except:
        print(d)
    
for i in range(0, 10):
    print(str(menu_info[i]))

1: 第一章 神秘的监狱 - 35935698.html - chapter_1.html
2: 第二章 韩东的发现 - 35935700.html - chapter_2.html
3: 第三章 从头开始 - 35935701.html - chapter_3.html
4: 第四章 上吊的青年 - 35935702.html - chapter_4.html
5: 第五章 便携式监狱 - 35935703.html - chapter_5.html
6: 第六章 祭典广场 - 35935704.html - chapter_6.html
7: 第七章 命运空间 - 35935705.html - chapter_7.html
8: 第八章 六人小队 - 35935706.html - chapter_8.html
9: 第九章 王婆 - 35935707.html - chapter_9.html
10: 第十章 噩梦 - 35935708.html - chapter_10.html


In [56]:
# {
#     'id': c, 
#     'link': f'chapter_{c}.html', 
#     'url':d.a['href'], 
#     'chapter':d.text
# }

manifest = []
ncx = []
navpoints = []
for m in menu_info:
    manifest.append(item % {'id': m.epub_link, 'url':m.epub_link})
    ncx.append(itemref % {'id': m.epub_link})
    navpoints.append(navpoint % (m.epub_link, m.id, m.chapter_title, m.epub_link))

In [57]:
manifest = '\n'.join(manifest)
ncx = '\n'.join(ncx)

In [58]:
with open(f'./{book_title}/OPS/content.opf', 'w', encoding="utf-8") as tmp:
    tmp.write(opfcontent % {
        'metadata': metadata,
        'manifest': manifest,
        'ncx': ncx,
    })

In [59]:
with open(f'./{book_title}/OPS/content.ncx', 'w', encoding="utf-8") as tmp:
    tmp.write(ncxcontent % {
        'title': book_title,
        'creator': author,
        'navpoints': '\n'.join(navpoints)
    })

## Download!

In [60]:
os.getcwd()

'C:\\Users\\jackz\\Developer\\FictionBot-IV'

In [61]:
cover_img = soup.find("img")
if cover_img:
    cover_img = cover_img['src']
    img = requests.get(cover_img, stream=True)
    if img.status_code == 200:
        with open(f"./{book_title}/OPS/cover.jpg", "wb") as f:
            shutil.copyfileobj(img.raw, f)
    del img

In [62]:
template = '''<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="zh-CN">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<link rel="stylesheet" type="text/css" href="css/main.css"/>
<title>%(title)s</title>
</head>
<body> c
<h2>%(title)s</h2>
<div>
%(content)s
</div>
</body>
</html>
'''

In [80]:
ch = menu_info[0]
print(ch.url)
source = requests.get(book_url + ch.url)
soup = BeautifulSoup(source.text, "html.parser")
sentences = soup.find("div", attrs={'id':'content'}).findAll(text=True)
contents = []
for s in sentences:
    tmp = s.replace('\xa0', '')
    contents.append(f'<p>{tmp}</p>')
if base_url in contents[0]:
    contents = contents[1:]


35935698.html


['<p>污水横流、菌斑肆掠。</p>',
 '<p> 某一废弃的监狱深处……</p>',
 '<p> 啪！</p>',
 '<p> 一团由质膜包裹的细胞团，竟从某具尸体的表面分离了出来。</p>',
 '<p> 细胞团体仅有人类食指大小，宏观外表就像一团白色鼻涕。</p>',
 '<p> 这一细胞团似乎具备着独立的思维与行动能力。</p>',
 '<p> 但由于神经系统的不完善性，并无五感。</p>',
 '<p> 不过，在细胞团与外物接触时，能通过‘细胞间信号传递’的特别手段，获取物质的基础信息。</p>',
 '<p> 分离出来的细胞没有过久的停留，开始移动了。</p>',
 '<p> 转录与翻译。</p>',
 '<p> 肌动蛋白产生。</p>',
 '<p> 这团手指头大小的细胞群体开始进行极为缓慢的‘迁移运动’，大约与蜗牛的移动速度相当。</p>',
 '<p> “不行……不够完美，也不是我想要的。”</p>',
 '<p> 细胞团似乎主动舍弃了这具看似完整且强壮的肉体，继续在监狱里展开搜寻……</p>',
 '<p> 若当前能有一束火把提供照明。</p>',
 '<p> 你会发现这一处巨型牢房中，由细胞团所舍弃的肉体达到上百具。</p>',
 '<p> …………</p>',
 '<p> 细胞团并非自主形成。</p>',
 '<p> 它有着自己的名字-韩东。</p>',
 '<p> 华侨，意大利佛罗伦萨大学生命科学院的副教授，在成为这团细胞凝聚体前，他刚好31岁。</p>',
 '<p> 于2018年7月21日，因肺癌死在佛罗伦萨中心医院的病床上。</p>',
 '<p> 病房里摆满着鲜花，然而这些鲜花却来源于他的学生，而非家人。</p>',
 '<p> 在死亡的一刻，病痛折磨消散一空，韩东反而感觉释然。</p>',
 '<p> 没有升入天堂或是堕入地狱，也没有喝下孟婆汤、走上来奈何桥、经历所谓的轮回转世。</p>',
 '<p> 迎接他的只有无尽黑暗而已。</p>',
 '<p> 然而，他的意识却在这一过程中始终存在。</p>',
 '<p> “我到底死没死？！人死后，大脑会继续保持活性五分钟……但我这已经死了一小时了吧？”</p>',
 '<p> 韩东的‘时间感’极强，意识清醒的他，很清楚自己‘死亡’了多长时间。</p>',
 '<p> 韩东慢慢试着

In [81]:
for ch in menu_info:
    t = ch.epub_link
    print("正在下载：" + t)
    source = requests.get(book_url + ch.url)
    soup = BeautifulSoup(source.text, "html.parser")
    sentences = soup.find("div", attrs={'id':'content'}).findAll(text=True)
    contents = []
    for s in sentences:
        tmp = s.replace('\xa0', '')
        contents.append(f'<p>{tmp}</p>')
    if base_url in contents[0]:
        contents = contents[1:]
#     source = requests.get(book_url + ch.url)
#     soup = BeautifulSoup(source.text, "html.parser")
#     sentences = soup.find("div", attrs={'id':'content'}).findAll(text=True)
#     contents = []
    with open(f'./{book_title}/OPS/{t}', 'w', encoding="utf-8") as f:
        f.write(template % {
            'title': ch.chapter_title,
            'content': '\n'.join(contents)
        })

正在下载：chapter_1.html
正在下载：chapter_2.html
正在下载：chapter_3.html
正在下载：chapter_4.html
正在下载：chapter_5.html
正在下载：chapter_6.html
正在下载：chapter_7.html
正在下载：chapter_8.html
正在下载：chapter_9.html
正在下载：chapter_10.html
正在下载：chapter_11.html
正在下载：chapter_12.html
正在下载：chapter_13.html
正在下载：chapter_14.html
正在下载：chapter_15.html
正在下载：chapter_16.html
正在下载：chapter_17.html
正在下载：chapter_18.html
正在下载：chapter_19.html
正在下载：chapter_20.html
正在下载：chapter_21.html
正在下载：chapter_22.html
正在下载：chapter_23.html
正在下载：chapter_24.html
正在下载：chapter_25.html
正在下载：chapter_26.html
正在下载：chapter_27.html
正在下载：chapter_28.html
正在下载：chapter_29.html
正在下载：chapter_30.html
正在下载：chapter_31.html
正在下载：chapter_32.html
正在下载：chapter_33.html
正在下载：chapter_34.html
正在下载：chapter_35.html
正在下载：chapter_36.html
正在下载：chapter_37.html
正在下载：chapter_38.html
正在下载：chapter_39.html
正在下载：chapter_40.html
正在下载：chapter_41.html
正在下载：chapter_42.html
正在下载：chapter_43.html
正在下载：chapter_44.html
正在下载：chapter_45.html
正在下载：chapter_46.html
正在下载：chapter_47.html
正在下载：chapter_48.html
正

正在下载：chapter_379.html
正在下载：chapter_380.html
正在下载：chapter_381.html
正在下载：chapter_382.html
正在下载：chapter_383.html
正在下载：chapter_384.html
正在下载：chapter_385.html
正在下载：chapter_386.html
正在下载：chapter_387.html
正在下载：chapter_388.html
正在下载：chapter_389.html
正在下载：chapter_390.html
正在下载：chapter_391.html
正在下载：chapter_392.html
正在下载：chapter_393.html
正在下载：chapter_394.html
正在下载：chapter_395.html
正在下载：chapter_396.html
正在下载：chapter_397.html
正在下载：chapter_398.html
正在下载：chapter_399.html
正在下载：chapter_400.html
正在下载：chapter_401.html
正在下载：chapter_402.html
正在下载：chapter_403.html
正在下载：chapter_404.html
正在下载：chapter_405.html
正在下载：chapter_406.html
正在下载：chapter_407.html
正在下载：chapter_408.html
正在下载：chapter_409.html
正在下载：chapter_410.html
正在下载：chapter_411.html
正在下载：chapter_412.html
正在下载：chapter_413.html
正在下载：chapter_414.html
正在下载：chapter_415.html
正在下载：chapter_416.html
正在下载：chapter_417.html
正在下载：chapter_418.html
正在下载：chapter_419.html
正在下载：chapter_420.html
正在下载：chapter_421.html
正在下载：chapter_422.html
正在下载：chapter_423.html
正在下载：chapt

正在下载：chapter_753.html
正在下载：chapter_754.html
正在下载：chapter_755.html
正在下载：chapter_756.html
正在下载：chapter_757.html
正在下载：chapter_758.html
正在下载：chapter_759.html
正在下载：chapter_760.html
正在下载：chapter_761.html
正在下载：chapter_762.html
正在下载：chapter_763.html
正在下载：chapter_764.html
正在下载：chapter_765.html
正在下载：chapter_766.html
正在下载：chapter_767.html
正在下载：chapter_768.html
正在下载：chapter_769.html
正在下载：chapter_770.html
正在下载：chapter_771.html
正在下载：chapter_772.html
正在下载：chapter_773.html
正在下载：chapter_774.html
正在下载：chapter_775.html
正在下载：chapter_776.html
正在下载：chapter_777.html
正在下载：chapter_778.html
正在下载：chapter_779.html
正在下载：chapter_780.html
正在下载：chapter_781.html
正在下载：chapter_782.html
正在下载：chapter_783.html
正在下载：chapter_784.html
正在下载：chapter_785.html
正在下载：chapter_786.html
正在下载：chapter_787.html
正在下载：chapter_788.html
正在下载：chapter_789.html
正在下载：chapter_790.html
正在下载：chapter_791.html
正在下载：chapter_792.html
正在下载：chapter_793.html
正在下载：chapter_794.html
正在下载：chapter_795.html
正在下载：chapter_796.html
正在下载：chapter_797.html
正在下载：chapt

正在下载：chapter_1121.html
正在下载：chapter_1122.html
正在下载：chapter_1123.html
正在下载：chapter_1124.html
正在下载：chapter_1125.html
正在下载：chapter_1126.html
正在下载：chapter_1127.html
正在下载：chapter_1128.html
正在下载：chapter_1129.html
正在下载：chapter_1130.html
正在下载：chapter_1131.html
正在下载：chapter_1132.html
正在下载：chapter_1133.html
正在下载：chapter_1134.html
正在下载：chapter_1135.html
正在下载：chapter_1136.html
正在下载：chapter_1137.html
正在下载：chapter_1138.html
正在下载：chapter_1139.html
正在下载：chapter_1140.html
正在下载：chapter_1141.html
正在下载：chapter_1142.html
正在下载：chapter_1143.html
正在下载：chapter_1144.html
正在下载：chapter_1145.html
正在下载：chapter_1146.html
正在下载：chapter_1147.html
正在下载：chapter_1148.html
正在下载：chapter_1149.html
正在下载：chapter_1150.html
正在下载：chapter_1151.html
正在下载：chapter_1152.html
正在下载：chapter_1153.html
正在下载：chapter_1154.html
正在下载：chapter_1155.html
正在下载：chapter_1156.html
正在下载：chapter_1157.html
正在下载：chapter_1158.html
正在下载：chapter_1159.html
正在下载：chapter_1160.html
正在下载：chapter_1161.html
正在下载：chapter_1162.html
正在下载：chapter_1163.html
正在下载：chapte

正在下载：chapter_1478.html
正在下载：chapter_1479.html
正在下载：chapter_1480.html
正在下载：chapter_1481.html
正在下载：chapter_1482.html
正在下载：chapter_1483.html
正在下载：chapter_1484.html
正在下载：chapter_1485.html
正在下载：chapter_1486.html
正在下载：chapter_1487.html
正在下载：chapter_1488.html
正在下载：chapter_1489.html
正在下载：chapter_1490.html
正在下载：chapter_1491.html
正在下载：chapter_1492.html
正在下载：chapter_1493.html
正在下载：chapter_1494.html
正在下载：chapter_1495.html
正在下载：chapter_1496.html
正在下载：chapter_1497.html
正在下载：chapter_1498.html
正在下载：chapter_1499.html
正在下载：chapter_1500.html
正在下载：chapter_1501.html
正在下载：chapter_1502.html
正在下载：chapter_1503.html
正在下载：chapter_1504.html
正在下载：chapter_1505.html
正在下载：chapter_1506.html
正在下载：chapter_1507.html
正在下载：chapter_1508.html
正在下载：chapter_1509.html
正在下载：chapter_1510.html
正在下载：chapter_1511.html
正在下载：chapter_1512.html
正在下载：chapter_1513.html
正在下载：chapter_1514.html
正在下载：chapter_1515.html
正在下载：chapter_1516.html
正在下载：chapter_1517.html
正在下载：chapter_1518.html
正在下载：chapter_1519.html
正在下载：chapter_1520.html
正在下载：chapte

正在下载：chapter_1835.html
正在下载：chapter_1836.html
正在下载：chapter_1837.html
正在下载：chapter_1838.html
正在下载：chapter_1839.html
正在下载：chapter_1840.html
正在下载：chapter_1841.html
正在下载：chapter_1842.html
正在下载：chapter_1843.html
正在下载：chapter_1844.html
正在下载：chapter_1845.html
正在下载：chapter_1846.html
正在下载：chapter_1847.html
正在下载：chapter_1848.html
正在下载：chapter_1849.html
正在下载：chapter_1850.html
正在下载：chapter_1851.html
正在下载：chapter_1852.html
正在下载：chapter_1853.html
正在下载：chapter_1854.html
正在下载：chapter_1855.html
正在下载：chapter_1856.html
正在下载：chapter_1857.html
正在下载：chapter_1858.html
正在下载：chapter_1859.html
正在下载：chapter_1860.html
正在下载：chapter_1861.html
正在下载：chapter_1862.html
正在下载：chapter_1863.html
正在下载：chapter_1864.html
正在下载：chapter_1865.html
正在下载：chapter_1866.html
正在下载：chapter_1867.html
正在下载：chapter_1868.html
正在下载：chapter_1869.html
正在下载：chapter_1870.html
正在下载：chapter_1871.html
正在下载：chapter_1872.html
正在下载：chapter_1873.html
正在下载：chapter_1874.html
正在下载：chapter_1875.html
正在下载：chapter_1876.html
正在下载：chapter_1877.html
正在下载：chapte

## Pack the EPub Book!

In this step, we will zip the folder then turn it into a \*.epub package.

In [82]:
# Collect all files in the folder
file_paths = []
for root, directories, files in os.walk(f'./{book_title}'): 
    for filename in files: 
        # join the two strings in order to form the full filepath. 
        filepath = os.path.join(root, filename) 
        file_paths.append(filepath)

In [83]:
from zipfile import ZipFile
with ZipFile(f"./{book_title}.epub", "w") as z:
    for f in file_paths:
        z.write(f)
        
print(f"Congratulations, {book_title}.epub has been freshly made!")

Congratulations, 我的细胞监狱.epub has been freshly made!


*Reference: https://www.jianshu.com/p/75b993cd2f68*
## The End